# Data Cleaning

## death_rates_smoking_age

In [2]:
import pandas as pd 
import numpy as np
import plotly as go
import pickle
from sklearn.linear_model import LinearRegression

In [6]:
death_rates_smoking_age = pd.read_csv('death_rates_smoking_age.csv')

In [32]:
death_rates_smoking_age_regions = death_rates_smoking_age[pd.isna(death_rates_smoking_age['Code'])]

In [33]:
death_rates_smoking_age_regions.to_csv('death_rates_smoking_age_regions.csv')

In [34]:
death_rates_smoking_age_country = death_rates_smoking_age.dropna()

In [36]:
death_rates_smoking_age_country.to_csv('death_rates_smoking_age_country.csv')

## stop_smoking

In [257]:
stop_smoking = pd.read_csv('stop_smoking.csv')

In [258]:
stop_smoking

,Entity,Code,Year,AvgCigarettePriceDollars,AvgTaxesAsPctCigarettePrice,EnforceBansTobaccoAd,HelpToQuit
0,Algeria,DZA,2012,1.84,47.0,4,3
1,Algeria,DZA,2014,2.09,40.9,4,4
2,Argentina,ARG,2012,2.79,69.9,4,4
3,Argentina,ARG,2014,3.47,69.7,4,5
4,Armenia,ARM,2012,1.69,38.4,2,4
...,...,...,...,...,...,...,...
769,Yemen,YEM,2010,NaN,NaN,4,3
770,Zambia,ZMB,2007,NaN,NaN,2,3
771,Zambia,ZMB,2010,NaN,NaN,2,3
772,Zimbabwe,ZWE,2007,NaN,NaN,2,3


## tobacco_production

In [259]:
tobacco_production = pd.read_csv('tobacco_production.csv')

In [260]:
tobacco_production

,Country or Area,Year,Unit,Value,Value Footnotes
0,Albania,2006,Metric tons,546.600000,NaN
1,Albania,2006,Mil. USD,1.324113,NaN
2,Albania,2005,Metric tons,1878.500000,NaN
3,Albania,2005,Mil. USD,4.844285,NaN
4,Albania,2004,Metric tons,751.900000,NaN
...,...,...,...,...,...
1203,Viet Nam,2004,Metric tons,12060.000000,NaN
1204,Viet Nam,2003,Metric tons,8817.000000,NaN
1205,Viet Nam,2002,Metric tons,6623.000000,NaN
1206,Viet Nam,2001,Metric tons,1944.000000,NaN


## Join to Predict Tax rate

In [261]:
tobacco_production_USD = tobacco_production[tobacco_production['Unit'] == 'Mil. USD']

In [262]:
tobacco_production_USD = tobacco_production_USD.drop(['Unit', 'Value Footnotes'], axis = 1)
tobacco_production_USD.rename(columns={'Country or Area':'Entity'}, inplace=True)

In [263]:
tobacco_production_USD

,Entity,Year,Value
1,Albania,2006,1.324113
3,Albania,2005,4.844285
5,Albania,2004,2.517025
29,Austria,2016,0.000000
31,Azerbaijan,2016,7.457441
...,...,...,...
1193,Uruguay,2000,5.543927
1194,Uruguay,1999,5.530012
1195,Uruguay,1998,5.304476
1196,Uruguay,1997,5.844921


In [264]:
tobacco_production_MT = tobacco_production[tobacco_production['Unit'] == 'Metric tons']

In [265]:
tobacco_production_MT = tobacco_production_MT.drop(['Unit', 'Value Footnotes'], axis = 1)
tobacco_production_MT.rename(columns={'Country or Area':'Entity'}, inplace=True)

In [266]:
tobacco_production_MT

,Entity,Year,Value
0,Albania,2006,546.6
2,Albania,2005,1878.5
4,Albania,2004,751.9
6,Algeria,2015,7483.0
7,Algeria,2014,7648.0
...,...,...,...
1203,Viet Nam,2004,12060.0
1204,Viet Nam,2003,8817.0
1205,Viet Nam,2002,6623.0
1206,Viet Nam,2001,1944.0


In [303]:
stop_smoking_join = pd.merge(stop_smoking,tobacco_production_MT,how = 'left' , on=['Entity', 'Year'])
#stop_smoking_join = stop_smoking_join.loc[:,~stop_smoking_join.columns.duplicated()]
stop_smoking_join

,Entity,Code,Year,AvgCigarettePriceDollars,AvgTaxesAsPctCigarettePrice,EnforceBansTobaccoAd,HelpToQuit,Value
0,Algeria,DZA,2012,1.84,47.0,4,3,8130.0
1,Algeria,DZA,2014,2.09,40.9,4,4,7648.0
2,Argentina,ARG,2012,2.79,69.9,4,4,NaN
3,Argentina,ARG,2014,3.47,69.7,4,5,NaN
4,Armenia,ARM,2012,1.69,38.4,2,4,NaN
...,...,...,...,...,...,...,...,...
769,Yemen,YEM,2010,NaN,NaN,4,3,NaN
770,Zambia,ZMB,2007,NaN,NaN,2,3,NaN
771,Zambia,ZMB,2010,NaN,NaN,2,3,NaN
772,Zimbabwe,ZWE,2007,NaN,NaN,2,3,NaN


### Join Sales per day

In [279]:
sales_per_day = pd.read_csv('sales_per_day.csv')

In [280]:
sales_per_day = sales_per_day.drop(['Entity'], axis = 1)

In [304]:
stop_smoking_join = pd.merge(stop_smoking_join,sales_per_day,how = 'left' , on=['Code', 'Year'])

In [305]:
stop_smoking_join

,Entity,Code,Year,AvgCigarettePriceDollars,AvgTaxesAsPctCigarettePrice,EnforceBansTobaccoAd,HelpToQuit,Value,amt
0,Algeria,DZA,2012,1.84,47.0,4,3,8130.0,NaN
1,Algeria,DZA,2014,2.09,40.9,4,4,7648.0,NaN
2,Argentina,ARG,2012,2.79,69.9,4,4,NaN,NaN
3,Argentina,ARG,2014,3.47,69.7,4,5,NaN,NaN
4,Armenia,ARM,2012,1.69,38.4,2,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...
769,Yemen,YEM,2010,NaN,NaN,4,3,NaN,NaN
770,Zambia,ZMB,2007,NaN,NaN,2,3,NaN,NaN
771,Zambia,ZMB,2010,NaN,NaN,2,3,NaN,NaN
772,Zimbabwe,ZWE,2007,NaN,NaN,2,3,NaN,NaN


### Join GDP per Cap


In [285]:
GDPperCap = pd.read_csv('GDP Per Capita.csv')

In [286]:
GDPperCap = GDPperCap.drop(['Country Name', 'Series Name', 'Series Code'], axis = 1)

In [287]:
year = ['1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020']

In [289]:
GDPperCap = GDPperCap.melt(id_vars=['Country Code'],value_vars = year, var_name="Year", value_name="GDP_USD")

In [292]:
GDPperCap.rename(columns={'Country Code':'Code'}, inplace=True)

In [293]:
GDPperCap['Year']=GDPperCap['Year'].astype(int)

In [306]:
stop_smoking_join = pd.merge(stop_smoking_join,GDPperCap,how = 'left' , on=['Code', 'Year'])

In [307]:
stop_smoking_join

,Entity,Code,Year,AvgCigarettePriceDollars,AvgTaxesAsPctCigarettePrice,EnforceBansTobaccoAd,HelpToQuit,Value,amt,GDP_USD
0,Algeria,DZA,2012,1.84,47.0,4,3,8130.0,NaN,5592.220115
1,Algeria,DZA,2014,2.09,40.9,4,4,7648.0,NaN,5493.056663
2,Argentina,ARG,2012,2.79,69.9,4,4,NaN,NaN,13082.66433
3,Argentina,ARG,2014,3.47,69.7,4,5,NaN,NaN,12334.79825
4,Armenia,ARM,2012,1.69,38.4,2,4,NaN,NaN,3681.844691
...,...,...,...,...,...,...,...,...,...,...
769,Yemen,YEM,2010,NaN,NaN,4,3,NaN,NaN,1334.784902
770,Zambia,ZMB,2007,NaN,NaN,2,3,NaN,NaN,1124.290586
771,Zambia,ZMB,2010,NaN,NaN,2,3,NaN,NaN,1489.459087
772,Zimbabwe,ZWE,2007,NaN,NaN,2,3,NaN,NaN,431.7872587


In [308]:
stop_smoking_join.rename(columns={'amt':'SalePerDay', 'Value':'ProductionMT'}, inplace=True)

In [309]:
stop_smoking_join.to_csv('stop_smoking_join.csv')

## Add Production USD

In [310]:
stop_smoking_join= pd.read_csv('stop_smoking_join.csv')

In [311]:
stop_smoking_join = pd.merge(stop_smoking_join,tobacco_production_USD,how = 'left' , on=['Entity', 'Year'])

In [312]:
stop_smoking_join.rename(columns={'Value':'ProductionUSD'}, inplace=True)

In [313]:
stop_smoking_join = stop_smoking_join.drop(['Unnamed: 0'], axis = 1)

In [316]:
stop_smoking_join.to_csv('stop_smoking_join.csv')

## Add Death_Rate

In [ ]:
stop_smoking_join= pd.read_csv('stop_smoking_join.csv')

In [370]:
death_rates_smokingPercent = pd.read_csv('death_rates_smokingPercent.csv')

In [371]:
death_rates_smokingPercent.drop(['Entity','Total Death Rate'],axis = 1, inplace = True)

In [372]:
stop_smoking_join = pd.merge(stop_smoking_join,death_rates_smokingPercent,how = 'left' , on=['Code', 'Year'])

In [375]:
stop_smoking_join.to_csv('stop_smoking_join.csv')

## Calculate Economical Benefits

In [376]:
stop_smoking_imputed = pd.read_csv('stop_smoking_imputed.csv')

In [352]:
def zStand(df, col_name_list):
    
    for column in col_name_list:
        df[column] = (df[column] - np.mean(df[column]))/np.std(df[column])
        
        
    return df

In [ ]:
zStand(stop_smoking_imputed, ['EconValue','InverseBan']).to_csv('stop_smoking_imputed.csv')

,Unnamed: 0,Entity,Code,Year,AvgCigarettePriceDollars,AvgTaxesAsPctCigarettePrice,EnforceBansTobaccoAd,HelpToQuit,ProductionMT,SalePerDay,GDP_USD,ProductionUSD,EconValue,InverseBan
0,0,Algeria,DZA,2012,1.840,47.00,4,3,8130.00000,8.52,5592.220115,107.316348,-0.480767,-0.719058
1,1,Algeria,DZA,2014,2.090,40.90,4,4,7648.00000,8.66,5493.056663,100.713852,-0.517851,-0.719058
2,2,Argentina,ARG,2012,2.790,69.90,4,4,7763.25989,6.40,13082.664330,95.835140,-0.599307,-0.719058
3,3,Argentina,ARG,2014,3.470,69.70,4,5,20069.96000,6.51,12334.798250,79.917662,-0.587376,-0.719058
4,4,Armenia,ARM,2012,1.690,38.40,2,4,23525.98910,8.91,3681.844691,128.014091,-0.402559,1.272055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,742,Venezuela,VEN,2007,5.020,65.94,2,4,24039.91000,4.29,8454.469536,236.821308,-0.504168,1.272055
737,743,Venezuela,VEN,2010,4.754,66.52,2,4,20028.06000,4.73,13825.357120,208.233128,-0.619923,1.272055
738,744,Venezuela,VEN,2014,4.622,66.52,2,4,20028.06000,5.14,16055.645320,208.233128,-0.645862,1.272055
739,745,Vietnam,VNM,2007,1.627,57.29,4,3,15690.57190,8.31,906.284230,154.382567,1.582571,-0.719058
